# SSEElementUpdater

> Flexible element update system for building out-of-band (OOB) swap elements. Register handlers by event type and compose updates without coupling to specific UI components.

In [ ]:
#| default_exp updater

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Callable, Dict, Any, List, Optional, Union

In [ ]:
#| export
class SSEElementUpdater:
    """Builds OOB swap elements without hardcoding UI components."""
    
    def __init__(self):
        """Initialize the updater with empty handler registry."""
        self._handlers: Dict[str, List[Callable]] = {}
        self._default_handler: Optional[Callable] = None
        self._preprocessors: List[Callable] = []
        self._postprocessors: List[Callable] = []
    
    def register(
        self,
        event_type: str,  # The event type to handle
        priority: int = 0  # Handler priority (higher numbers run first)
    ): # Decorator function
        """Decorator to register an update handler for a specific event type."""
        def decorator(
            handler: Callable  # The handler function to register
        ):
            if event_type not in self._handlers:
                self._handlers[event_type] = []
            
            # Add handler with priority
            self._handlers[event_type].append((priority, handler))
            # Sort by priority (descending)
            self._handlers[event_type].sort(key=lambda x: x[0], reverse=True)
            
            return handler
        return decorator
    
    def register_handler(
        self,
        event_type: str,  # The event type to handle
        handler: Callable,  # The handler function
        priority: int = 0  # Handler priority (higher numbers run first)
    ):
        """Register an update handler programmatically."""
        if event_type not in self._handlers:
            self._handlers[event_type] = []
        
        self._handlers[event_type].append((priority, handler))
        self._handlers[event_type].sort(key=lambda x: x[0], reverse=True)
    
    def set_default_handler(
        self,
        handler: Callable  # The default handler function
    ):
        """Set a default handler for unregistered event types."""
        self._default_handler = handler
    
    def add_preprocessor(
        self,
        processor: Callable  # Function that processes (event_type, data) and returns modified data
    ):
        """Add a preprocessor that runs before handlers."""
        self._preprocessors.append(processor)
    
    def add_postprocessor(
        self,
        processor: Callable  # Function that processes elements list and returns modified elements
    ):
        """Add a postprocessor that runs after handlers."""
        self._postprocessors.append(processor)
    
    def create_elements(
        self,
        event_type: str,  # The type of event
        data: Dict[str, Any]  # Event data
    ) -> List[Any]:  # List of elements to be sent via SSE
        """Create elements for a given event type and data."""
        # Run preprocessors
        processed_data = data
        for processor in self._preprocessors:
            processed_data = processor(event_type, processed_data)
        
        elements = []
        
        # Get handlers for this event type
        handlers = self._handlers.get(event_type, [])
        
        if handlers:
            # Run all registered handlers
            for priority, handler in handlers:
                result = handler(processed_data)
                if result:
                    if isinstance(result, list):
                        elements.extend(result)
                    else:
                        elements.append(result)
        elif self._default_handler:
            # Use default handler if no specific handlers registered
            result = self._default_handler(event_type, processed_data)
            if result:
                if isinstance(result, list):
                    elements.extend(result)
                else:
                    elements.append(result)
        
        # Run postprocessors
        for processor in self._postprocessors:
            elements = processor(elements)
        
        return elements
    
    def clear_handlers(
        self,
        event_type: Optional[str] = None  # Optional specific event type to clear
    ):
        """Clear handlers for a specific event type or all handlers."""
        if event_type:
            self._handlers.pop(event_type, None)
        else:
            self._handlers.clear()
    
    def get_registered_events(
        self
    ) -> List[str]:  # List of event types with registered handlers
        """Get list of registered event types."""
        return list(self._handlers.keys())

In [ ]:
element_updater = SSEElementUpdater()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()